# Checking statistical test

We want to check several things:
- the CHM implementation
- the minimal p-value computation
- the envelop computation

For this, we can use simple settings where we know the truth without needing the computational tricks, i.e where we can brute force compute all possible p-values.

## Setting

In [1]:
from importlib import reload
import sys
import os
sys.path.append(os.path.abspath("/Users/hector/Documents/BGWAS2/COIN/Scripts"))
import numpy as np
import random

In [2]:
import Envelope
reload(Envelope)

<module 'Envelope' from '/Users/hector/Documents/BGWAS2/COIN/Scripts/Envelope.py'>

## CMH test

In [3]:
from statsmodels.stats.contingency_tables import StratifiedTable
n = 15
K = 2
pheno = np.random.choice([0, 1], size=(n), p = [.7, .3])
pattern = np.random.choice([0, 1], size=(n), p = [.4, .6])
pop = np.random.choice(np.arange(K), size=(n), p = np.ones(K) / K)

We first compute the actual p-value

In [4]:
tables = np.zeros((2, 2, K))
for k in range(K):
    phenoPop = pheno[np.where(pop == k)]
    patternPop = pattern[np.where(pop == k)]
    tables[0, 0, k] = len(np.where(phenoPop[np.where(patternPop == 1)] == 1)[0])
    tables[0, 1, k] = len(np.where(phenoPop[np.where(patternPop == 0)] == 1)[0])
    tables[1, 0, k] = len(np.where(phenoPop[np.where(patternPop == 1)] == 0)[0])
    tables[1, 1, k] = len(np.where(phenoPop[np.where(patternPop == 0)] == 0)[0])

In [5]:
pvalue = StratifiedTable(tables).test_null_odds().pvalue

We then compute the p-value from our method

In [6]:
a = tables[0, 0, :]
xs = np.sum(tables, axis = 0)[0, :]
n1s = np.sum(tables, axis = 1)[0, :]
n2s = np.sum(tables, axis = 1)[1, :]
pvalue2 = Envelope.chi2.sf(Envelope.Th(a.sum(), xs, n1s, n2s), 1)

And finally, we can compare the twos

In [7]:
assert((pvalue - pvalue2) / pvalue2 < 10**(-10))

## Minimal p-value

We will first do a brute force computation, and check whether the value we estimate is indeed the true minimal p-value.

In [8]:
minP = Envelope.chi2.sf(Envelope.minimal_p_value(xs, n1s, n2s), 1)

In [9]:
a0s = range(int(max(0, xs[0] - n2s[0])),
            int(min(xs[0], n1s[0])) + 1)
a1s = range(int(max(0, xs[1] - n2s[1])),
            int(min(xs[1], n1s[1])) + 1)
minPbrute = 1
tables2 = np.zeros((2, 2, K))
for a0 in a0s:
    for a1 in a1s:
        tables2[0, 0, 0] = a0
        tables2[1, 0, 0] = xs[0] - a0
        tables2[0, 1, 0] = n1s[0] - a0
        tables2[1, 1, 0] = n2s[0] + a0 - xs[0]
        tables2[0, 0, 1] = a1
        tables2[1, 0, 1] = xs[1] - a1
        tables2[0, 1, 1] = n1s[1] - a1
        tables2[1, 1, 1] = n2s[1] + a1 - xs[1]
        pvalue = StratifiedTable(tables2).test_null_odds().pvalue
        if pvalue < minPbrute:
            minPbrute = pvalue

In [10]:
assert(((minP - minPbrute) / minPbrute) < 10**(-10)) 

## Envelope

Finally, we want to compute the envelope and make sure that it indeed fit the definition. 

In [11]:
n1s = np.array([4, 4])
n2s = np.array([5, 5])
Xs = np.array([6, 3])

In [12]:
minP = Envelope.chi2.sf(Envelope.envelope(Xs, n1s, n2s), 1)

In [13]:
ns = n1s + n2s
x0s = range(Xs[0], ns[0] + 1)
x1s = range(Xs[1], ns[1] + 1)
minPbrute = 1
for x0 in x0s:
    for x1 in x1s:
        xs = np.array([x0, x1])
        a0s = range(int(max(0, xs[0] - n2s[0])),
                    int(min(xs[0], n1s[0])) + 1)
        a1s = range(int(max(0, xs[1] - n2s[1])),
                    int(min(xs[1], n1s[1])) + 1) 
        tables2 = np.zeros((2, 2, K))
        for a0 in a0s:
            for a1 in a1s:
                tables2[0, 0, 0] = a0
                tables2[1, 0, 0] = xs[0] - a0
                tables2[0, 1, 0] = n1s[0] - a0
                tables2[1, 1, 0] = n2s[0] + a0 - xs[0]
                tables2[0, 0, 1] = a1
                tables2[1, 0, 1] = xs[1] - a1
                tables2[0, 1, 1] = n1s[1] - a1
                tables2[1, 1, 1] = n2s[1] + a1 - xs[1]
                pvalue = StratifiedTable(tables2).test_null_odds().pvalue
                if pvalue < minPbrute:
                    minPbrute = pvalue

/usr/local/lib/python3.8/site-packages/statsmodels/stats/contingency_tables.py:1049: RuntimeWarning: invalid value encountered in double_scalars
  statistic /= denom


In [14]:
assert(((minP - minPbrute) / minPbrute) < 10**(-10)) 

### Comparison with the old envelope

In [15]:
n1s = np.array([4, 4])
n2s = np.array([5, 5])
xs = np.array([7, 6])
Env = Envelope.envelope(xs, n1s, n2s)
Old_Env = minP = Envelope.old_envelope(xs, n1s, n2s)
assert(((Env - Old_Env) / Env) < 10**(-10))
n1s = np.array([4, 4])
n2s = np.array([5, 5])
xs = np.array([7, 5])
Env = Envelope.envelope(xs, n1s, n2s)
Old_Env = minP = Envelope.old_envelope(xs, n1s, n2s)
assert(((Env - Old_Env) / Env) < 10**(-10)) 
n1s = np.array([4, 4])
n2s = np.array([5, 5])
xs = np.array([7, 4])
Env = Envelope.envelope(xs, n1s, n2s)
Old_Env = minP = Envelope.old_envelope(xs, n1s, n2s)
assert(Env > Old_Env)

In [16]:
n1s = np.array([212, 10])
n2s = np.array([194, 12])
xs = np.array([218, 10])
Env = Envelope.envelope(xs, n1s, n2s)
Old_Env = minP = Envelope.old_envelope(xs, n1s, n2s)
assert(Env >= Old_Env)